# Getting Started with Miyagi's Retrieval Augmented Generation (RaG) Workflow using Azure Cognitive Search (ACS) and Semantic Kernel

To quickly get started, follow these steps:

1. Ensure the [Polyglot notebooks extension](https://marketplace.visualstudio.com/items?itemName=ms-dotnettools.dotnet-interactive-vscode) is installed.
2. [Create a new Azure OpenAI service (or use an existing OpenAI service)](https://learn.microsoft.com/en-us/azure/ai-services/openai/chatgpt-quickstart?tabs=command-line&pivots=programming-language-studio#prerequisites).
3. [Deploy the `gpt-35-turbo` and `text-embeddings-ada-002` models](https://learn.microsoft.com/en-us/azure/ai-services/openai/concepts/models#working-with-models).
4. [Create an Azure Cognitive Search instance and enable the Semantic Search capability](https://learn.microsoft.com/en-us/azure/search/semantic-search-overview#enable-semantic-search).
5. Copy the `.env.example` file from the `rag` folder to `dotnet/.env` and paste the corresponding values from the resources you provisioned in the earlier steps.
6. Click on `Run All`.

> You will need an [.Net 7 SDK](https://dotnet.microsoft.com/en-us/download) and [Polyglot](https://marketplace.visualstudio.com/items?itemName=ms-dotnettools.dotnet-interactive-vscode) to get started with this notebook using .Net Interactive

## Overview
In this notebook, we will show you how to quickly get started with Miyagi's Retrieval Augmented Generation (RaG) workflow using Semantic Kernel, Azure Cognitive Search (ACS) and Cosmos. Through this use case, involving stock recommendations, you will understand the basics of RaG and the capabilities of Semantic Kernel, Azure Cognitive Search and Cosmos to adapt to your specific use case.

## Load settings from .env file

In [2]:
#r "nuget: dotenv.net"
dotenv.net.DotEnv.Load();
var env = dotenv.net.DotEnv.Read();

Installed Packages dotenv.net, 3.1.3

## Prepare the kernel with memory using Azure Cognitive Search

We first add the nuget packages (dependecies) required to instantiate the kernel. 

In [3]:
#r "nuget: Microsoft.SemanticKernel, 1.0.1"
#r "nuget: Microsoft.SemanticKernel.Connectors.AzureAISearch, 1.0.1-alpha"
#r "nuget: Microsoft.SemanticKernel.Connectors.MongoDB, 1.0.1-alpha"
#r "nuget: Microsoft.SemanticKernel.Connectors.Postgres, 1.0.1-alpha"
#r "nuget: Microsoft.SemanticKernel.Plugins.OpenApi, 1.0.1-alpha"
#r "nuget: Microsoft.SemanticKernel.Plugins.Memory, 1.0.1-alpha"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core, 1.0.1-alpha"
#r "nuget: Microsoft.SemanticKernel.PromptTemplates.Handlebars, 1.0.1"
#r "nuget:  Microsoft.SemanticKernel.Yaml, 1.0.1"
#r "nuget:  MongoDB.Bson, 2.22.0"
#r "nuget:  MongoDB.Driver, 2.22.0"


Installed Packages Microsoft.SemanticKernel, 1.0.1 Microsoft.SemanticKernel.Connectors.AzureAISearch, 1.0.1-alpha Microsoft.SemanticKernel.Connectors.MongoDB, 1.0.1-alpha Microsoft.SemanticKernel.Connectors.Postgres, 1.0.1-alpha Microsoft.SemanticKernel.Plugins.Core, 1.0.1-alpha Microsoft.SemanticKernel.Plugins.Memory, 1.0.1-alpha Microsoft.SemanticKernel.Plugins.OpenApi, 1.0.1-alpha Microsoft.SemanticKernel.PromptTemplates.Handlebars, 1.0.1 Microsoft.SemanticKernel.Yaml, 1.0.1 MongoDB.Bson, 2.22.0 MongoDB.Driver, 2.22.0

In [4]:
using Microsoft.SemanticKernel;

With dependencies installed and configured, we create an instance of the kernel using the built-in `Kernel.Builder` class

In [5]:
var builder = Kernel.CreateBuilder();
        builder.Services.AddAzureOpenAIChatCompletion(
            env["AZURE_OPENAI_CHAT_MODEL"],
            env["AZURE_OPENAI_ENDPOINT"],
            env["AZURE_OPENAI_API_KEY"]
        );

var kernel = builder.Build();

To make the kernel stateful, we will use Memory Plugin with a Azure Cognitive Search and Embedding backend. 

In [6]:
using Microsoft.SemanticKernel.Connectors.AzureAISearch;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.Memory;
using Microsoft.SemanticKernel.Plugins.Core;
using Microsoft.SemanticKernel.Connectors.MongoDB;
//using Microsoft.SemanticKernel.Connectors.Postgres;
//using Npgsql;
  
#pragma warning disable SKEXP0003
#pragma warning disable SKEXP0011
#pragma warning disable SKEXP0021
#pragma warning disable SKEXP0030


//NpgsqlDataSourceBuilder dataSourceBuilder = new(env["AZURE_COSMOS_MONGO_VECTORDB_CONNECTIONSTRING"]);
//dataSourceBuilder.UseVector();
//NpgsqlDataSource dataSource = dataSourceBuilder.Build();
#pragma warning disable SKEXP0032
//IMemoryStore store = new PostgresMemoryStore(dataSource, vectorSize: 1536, schema: "public");

IMemoryStore store = new MongoDBMemoryStore(env["AZURE_COSMOS_MONGO_VECTORDB_CONNECTIONSTRING"], env["AZURE_COSMOS_MONGO_VECTORDB_NAME"]);
// IMemoryStore store = new AzureAISearchMemoryStore(
//     env["AZURE_COGNITIVE_SEARCH_ENDPOINT"],
//     env["AZURE_COGNITIVE_SEARCH_API_KEY"]);
            
// Create an embedding generator to use for semantic memory.
var embeddingGenerator = new AzureOpenAITextEmbeddingGenerationService(
    env["AZURE_OPENAI_EMBEDDING_MODEL"],
    env["AZURE_OPENAI_ENDPOINT"],
    env["AZURE_OPENAI_API_KEY"],
    env["AZURE_OPENAI_EMBEDDING_MODEL"]);
SemanticTextMemory textMemory = new(store, embeddingGenerator);

## Vectorize and persist embeddings in Azure Cognitive Search with Semantic Kernel

Embeddings are a type of data transformation that convert text into high-dimensional vectors that capture semantic relationships between words or text chunks. This vectorized form allows machines to process text in a manner akin to numerical data, enabling operations like similarity comparison which is foundational for vector databases to retrieve meaningful context for LLMs. Embeddings are essential to generate and retrieve relevant content in a RaG workflow because:

1. They normalize your data in a common mathematical representation, making it machine-friendly
1. They compress data, retaining essential representations while reducing dimensionality
1. They preserve relationships within your data
1. They are representationally dense, meaning they are more efficient to store and process


> See a visual representation of the embeddings in 2D space below (from Cohere):

![Semantic Search](https://txt.cohere.com/content/images/2023/01/Vis-7-2.jpg)

Now, let’s delve into the process of creating and utilizing embeddings in the context of ACS.

In this scenario, we have a snippet of proprietary text data—extracted from a book—which needs to be processed and indexed for semantic search. The steps involve:

1. Chunking the Text: Breaking down the text into manageable pieces, referred to as 'chunks'.
1. Generating Embeddings: Transforming these chunks into high-dimensional vectors using an Azure Text Embedding Generation Service.
1. Persisting Embeddings to ACS: Storing these embeddings in Azure Cognitive Search to facilitate semantic search.

The provided code snippet orchestrates these steps. It reads the text data from a local file, chunks it, generates embeddings for each chunk, and then persists these embeddings to ACS. The specific operations are abstracted for simplicity using methods like SaveInformationAsync, which is part of a hypothetical kernel object in this code. The end result is a collection of indexed embeddings in ACS that significantly enhance search quality by moving beyond mere keyword matching to understanding semantic relationships.

In [7]:
using Microsoft.SemanticKernel.Text;
using System.IO;

#pragma warning disable SKEXP0055

var dataset = "intelligent-investor.txt";
var recommendationServicePath = "../../../../services/recommendation-service/dotnet";
const int MaxTokensPerParagraph = 300;
const int MaxTokensPerLine = 100;

// Load text data from a local file
var filePath = Path.Combine(recommendationServicePath, "resources", "sample-datasets", dataset);
var streamReader = new StreamReader(filePath);
var text = await streamReader.ReadToEndAsync();

// Define a memory collection name for indexing
var memoryCollectionName = "miyagi-investment-thesis";

// Chunk the text into lines and paragraphs for manageable processing
var lines = TextChunker.SplitPlainTextLines(text, MaxTokensPerLine);
var chunks = TextChunker.SplitPlainTextParagraphs(lines, MaxTokensPerParagraph);

// Loop through each chunk to generate embeddings and persist to ACS
for (var i = 0; i < chunks.Count; i++)
{
    var chunk = chunks[i];
    System.Console.WriteLine($"{chunk}");
    // Generate an identifier for each chunk
    await textMemory.SaveInformationAsync(
        memoryCollectionName,
        chunk,
        $"{dataset}-{i}",
        $"Dataset: {dataset} Chunk: {i}",
        i.ToString());
}

// Output the result to console
System.Console.WriteLine($"Saved {chunks.Count} chunks to memory collection {memoryCollectionName}");


SYNOPSIS
This book will not teach you how to beat the market. However, it will teach you how to reduce risk, protect your capital from loss and reliably generate sustainable returns over the long run. Warren Buffett calls the Intelligent Investor ""by far the best book on investing ever written.""
Benjamin's proven value investing approach replaces risky attempts to project future share prices with sound investments based on the underlying value of the company's tangible assets.
The Intelligent Investor by Benjamin Graham gives you everything you need to equip yourself with the investor's mindset necessary to avoid the panic of market fluctuations that plague the ordinary investor. Don't be ordinary. Be intelligent.

TOP 20 INSIGHTS
There are two kinds of investors. Defensive investors aim to protect their capital from losses, generate decent returns and minimize frequent decisions. Enterprising investors devote most of their time to manage their portfolios actively. An enterprising in

## Search and retrieve documents using Semantic Kernel

To search and retrieve documents, we will use the `SearchAsync` method of Semantic memory that encapsulates retrieval abstractions of different memory providers. This method takes in a query and returns a list of documents that are relevant to the query.

In [15]:
using MongoDB.Bson;
using MongoDB.Driver;
using System.Linq;
using System.Text.Json;

var query = "Ben Graham's investment philosophy";

MongoClient _client = new MongoClient(env["AZURE_COSMOS_MONGO_VECTORDB_CONNECTIONSTRING"]);
IMongoDatabase _database = _client.GetDatabase(env["AZURE_COSMOS_MONGO_VECTORDB_NAME"]);
int _maxVectorSearchResults = 2;
 
var queryEmbedding = await embeddingGenerator.GenerateEmbeddingsAsync([query], null, default);
var jsonString = JsonSerializer.Serialize(queryEmbedding[0]);
var embedding = JsonSerializer.Deserialize<List<float>>(jsonString);
 
var indexName = "vectorSearchIndex";
 
string jsonDoc = "{\"name\": \"" + indexName + "\",\"key\": {\"embedding\": \"cosmosSearch\"}, \"cosmosSearchOptions\": { \"kind\": \"vector-ivf\", \"numLists\": 2, \"similarity\": \"COS\", \"dimensions\": 1536 } }";
 
var collection = _database.GetCollection<BsonDocument>("miyagi-investment-thesis");
try
{
    _database.RunCommand<BsonDocument>("{ \"createIndexes\":\"miyagi-investment-thesis\",indexes:[" + jsonDoc + "]}");
}
catch (Exception e)
{
    // collection.Indexes.DropOne(indexName);
    // _database.RunCommand<BsonDocument>("{ \"createIndexes\":\"miyagi-investment-thesis\",indexes:[" + jsonDoc + "]}");
}
 
var embeddingsArray = new BsonArray(embedding.Select(e => new BsonDouble(Convert.ToDouble(e))));
 
BsonDocument[] pipeline = new BsonDocument[]
                {
                    BsonDocument.Parse($"{{$search: {{cosmosSearch: {{ vector: [{string.Join(',', embeddingsArray)}], path: 'embedding', k: {_maxVectorSearchResults}}}, returnStoredSource:true}}}}"),
                    BsonDocument.Parse($"{{$project: {{similarityScore: {{\"$meta\": \"searchScore\" }},document:\"$$ROOT\"}}}}"),
                };
 
List<BsonDocument> bsonDocuments = await collection.Aggregate<BsonDocument>(pipeline).ToListAsync();

foreach(var result in bsonDocuments){
    Console.WriteLine("   " + result["document"]["metadata"]["text"]);
    Console.WriteLine("   Relevance: " + result["similarityScore"] + "\n");
}
 
//var resultDocuments = string.Join(" ", result);

[-0.01922113,-0.027106723,0.018781563,-0.032235023,0.0009682162,0.01602427,-0.03535196,0.0016125836,-0.018275393,-0.0017699292,-0.017476177,0.022564515,0.011881669,0.017969027,0.016663643,0.024269508,0.029144723,0.002798919,0.0155447405,-0.008871292,-0.029757455,-0.0061805993,0.011502042,-0.028079102,-0.004089319,0.0010864335,0.025574895,-0.012720846,-0.004868554,0.0061805993,0.036950395,2.660149E-05,-0.01031654,0.022950804,-0.014892047,-0.015864426,-0.008305181,0.026214266,0.048006207,-0.004089319,0.0031735513,0.0010339849,-0.0033200742,-0.0259745,0.00084666885,0.0025641494,0.02072632,-0.0029271266,-0.018781563,-0.005994116,0.018874805,0.025761377,-0.033860095,0.01401291,0.0061872597,0.008544945,0.015571381,0.004352394,0.008897932,-0.0072662,0.007945534,-0.010962572,-0.009637206,0.022551196,-0.012174715,0.008917912,0.00024871418,0.0082718795,0.0124744205,0.0022910843,0.027599573,0.029544331,0.004345734,0.012814088,0.012993911,-0.00775905,-0.015598021,-0.0049085147,-0.0003248894,0.0076

## Grounding with Native Functions and SK's Memory "recall"

> Note that this prompt template (semantic function) is located under services/reccommendation-service/dotnet/plugins/AdvisorPlugin

In [ ]:
using System.IO;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.PromptTemplates.Handlebars;

var pluginFolder = $"{recommendationServicePath}/Resources/Prompts";

// Construct the full file path
var filePath = Path.Combine(pluginFolder, "InvestmentAdvise.prompt.yaml");

KernelFunction advisorPlugin;

// Use StreamReader to read the file
using (StreamReader reader = new StreamReader(filePath))
{
    // Create the function from the YAML file using Handlebars template
    advisorPlugin = kernel.CreateFunctionFromPromptYaml(
        reader.ReadToEnd(),
        promptTemplateFactory: new HandlebarsPromptTemplateFactory()
    );
}

In [ ]:
using Microsoft.SemanticKernel.Plugins.Memory;

#pragma warning disable SKEXP0052

// recall is from the TextMemoryPlugin, which has a recall method that can be invoked from Semantic Function to perform retrieval
KernelPlugin memoryPlugin = kernel.ImportPluginFromObject(new TextMemoryPlugin(textMemory));

### Set Kernel Arguments (context variables)

In [ ]:
using System.Text.Json;
using Microsoft.SemanticKernel.ChatCompletion;

#pragma warning disable SKEXP0052

// Create few-shot examples
List<ChatHistory> fewShotExamples = [
    [
        new ChatMessageContent(AuthorRole.User, @"{""stocks"":[{""symbol"":""MSFT"",""allocation"":0.6},{""symbol"":""ACN"",""allocation"":0.4}]}"),
        new ChatMessageContent(AuthorRole.Assistant, @"{""portfolio"":[{""symbol"":""MSFT"",""gptRecommendation"":""Booyah! Hold on, steady growth! Diversify, though!""},{""symbol"":""ACN"",""gptRecommendation"":""Buy! Services will see a boom!""}]}")
    ]
];

/*
    * Resolve issue with Handlebars ProcessPositionalArguments issue with parameter parsing for TextMemoryPlugin-recall
    * In prompt template for handlebars, the following is not working:
    * {{TextMemoryPlugin-recall semanticQuery, miyagi-embeddings, 0.8, 3}}
    *
    * Meanwhile, use this alternative approach
*/
var memories = await kernel.InvokeAsync(memoryPlugin["Recall"], new()
{
    [TextMemoryPlugin.InputParam] = $"Investment advise for aggressive risk level",
    [TextMemoryPlugin.CollectionParam] = "miyagi-investment-thesis",
    [TextMemoryPlugin.LimitParam] = "2",
    [TextMemoryPlugin.RelevanceParam] = "0.8",
});

Console.WriteLine($"Memories: {memories.GetValue<string>()}");
Console.WriteLine();

// Set harcoded parameters for the AdvisorPlugin
var stocks = new[] {
    new {symbol = "MSFT", allocation = 0.3},
    new {symbol = "ACN", allocation = 0.1},
    new {symbol = "JPM", allocation = 0.3},
    new {symbol = "PEP", allocation = 0.3}
};

var arguments = new KernelArguments
{
    ["userId"] = "50",
    ["stocks"] = JsonSerializer.Serialize(stocks),
    ["risk"] = "aggressive",
    ["fewShotExamples"] = fewShotExamples,
    ["voice"] = "Jim Cramer",
    ["memories"] = memories.GetValue<string>()
};
arguments

### Create native function

In [ ]:
// Copyright (c) Microsoft. All rights reserved.

using System.ComponentModel;
using Microsoft.SemanticKernel;


/// <summary>
///     UserProfilePlugin shows a native skill example to look user info given userId.
/// </summary>
/// <example>
///     Usage: kernel.ImportSkill("UserProfilePlugin", new UserProfilePlugin());
///     Examples:
///     SKContext["userId"] = "000"
///     {{UserProfilePlugin.GetUserAge $userId }} => {userProfile}
/// </example>
public class UserProfilePlugin
{
    /// <summary>
    ///     Name of the context variable used for UserId.
    /// </summary>
    public const string UserId = "UserId";

    private const string DefaultUserId = "40";
    private const int DefaultAnnualHouseholdIncome = 150000;
    private const int Normalize = 81;

    /// <summary>
    ///     Lookup User's age for a given UserId.
    /// </summary>
    /// <example>
    ///     SKContext[UserProfilePlugin.UserId] = "000"
    /// </example>
    [KernelFunction, Description("Given a userId, get user age")]
    public string GetUserAge(
        [Description("Unique identifier of a user")]
        string userId)
    {
        // userId = context.Variables.ContainsKey(UserId) ? context[UserId] : DefaultUserId;
        userId = string.IsNullOrEmpty(userId) ? DefaultUserId : userId;
        
        int age;

        if (int.TryParse(userId, out var parsedUserId))
            age = parsedUserId > 100 ? parsedUserId % Normalize : parsedUserId;
        else
            age = int.Parse(DefaultUserId);

        // invoke a service to get the age of the user, given the userId
        return age.ToString();
    }

    /// <summary>
    ///     Lookup User's annual income given UserId.
    /// </summary>
    /// <example>
    ///     SKContext[UserProfilePlugin.UserId] = "000"
    /// </example>
    [KernelFunction, Description("Given a userId, get user annual household income")]
    public string GetAnnualHouseholdIncome(
        [Description("Unique identifier of a user")]
        string userId)
    {
        
        // userId = context.Variables.ContainsKey(UserId) ? context[UserId] : DefaultUserId;
        userId = string.IsNullOrEmpty(userId) ? DefaultUserId : userId;

        var random = new Random();
        var randomMultiplier = random.Next(1000, 8000);

        // invoke a service to get the annual household income of the user, given the userId
        var annualHouseholdIncome = int.TryParse(userId, out var parsedUserId)
            ? parsedUserId * randomMultiplier
            : DefaultAnnualHouseholdIncome;

        return annualHouseholdIncome.ToString();
    }
}

In [ ]:
// import the UserProfilePlugin
kernel.Plugins.AddFromType<UserProfilePlugin>();

### Invoke the LLM

In [ ]:
var result = await kernel.InvokeAsync(
            advisorPlugin,
            arguments
        );
result.GetValue<string>()

![RaG Workflow](../../../../assets/images/sk-memory-orchestration.png)